# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,-0.21,69,20,1.03,AR,1722879065
1,1,margaret river,-33.9500,115.0667,16.11,84,64,7.45,AU,1722879159
2,2,coahuayana de hidalgo,18.7000,-103.6583,33.01,53,72,2.41,MX,1722879160
3,3,conde,-7.2597,-34.9075,28.70,51,40,7.20,BR,1722878811
4,4,shalkar,47.8288,59.6258,31.77,25,62,6.97,KZ,1722879162


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = 0.5,
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df[(city_data_df['Max Temp'] <= 18) & (city_data_df['Max Temp'] >= 10) & (city_data_df['Wind Speed'] < 3) & (city_data_df['Cloudiness'] == 100)]

# Drop any rows with null values
ideal_cities_clean = ideal_cities.dropna()

# Display sample data
ideal_cities_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,novodvinsk,64.4165,40.8122,13.86,88,100,1.78,RU,1722879211
49,49,korem,12.5058,39.5228,15.51,82,100,0.37,ET,1722879215
63,63,prince rupert,54.3161,-130.3201,14.83,96,100,2.06,CA,1722879235
76,76,new norfolk,-42.7826,147.0587,10.51,80,100,2.27,AU,1722879250
81,81,yelizovo,53.1873,158.3805,12.08,100,100,1.00,RU,1722879256


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_clean[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
45,novodvinsk,RU,64.4165,40.8122,88,
49,korem,ET,12.5058,39.5228,82,
63,prince rupert,CA,54.3161,-130.3201,96,
76,new norfolk,AU,-42.7826,147.0587,80,
81,yelizovo,RU,53.1873,158.3805,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    #"filter":filters,
    "limit":limit,
    #"bias":bias,
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
novodvinsk - nearest hotel: Уют
korem - nearest hotel: No hotel found
prince rupert - nearest hotel: Crest Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
yelizovo - nearest hotel: Николина Усадьба
selfoss - nearest hotel: Miðgarður
nemuro - nearest hotel: お宿エクハシ
ola - nearest hotel: No hotel found
gornozavodsk - nearest hotel: Урал
hola - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
lebu - nearest hotel: No hotel found
dzuunharaa - nearest hotel: No hotel found
berdigestyakh - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
yuzhno-kurilsk - nearest hotel: Айсберг
novyy urgal - nearest hotel: No hotel found
nikolayevsk-on-amure - nearest hotel: Север


,City,Country,Lat,Lng,Humidity,Hotel Name
45,novodvinsk,RU,64.4165,40.8122,88,Уют
49,korem,ET,12.5058,39.5228,82,No hotel found
63,prince rupert,CA,54.3161,-130.3201,96,Crest Hotel
76,new norfolk,AU,-42.7826,147.0587,80,Woodbridge on the Derwent
81,yelizovo,RU,53.1873,158.3805,100,Николина Усадьба
83,selfoss,IS,63.9331,-20.9971,89,Miðgarður
119,nemuro,JP,43.3236,145.5750,100,お宿エクハシ
171,ola,RU,59.5833,151.2833,88,No hotel found
187,gornozavodsk,RU,58.3758,58.3211,97,Урал
200,hola,PL,51.5377,23.1902,95,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)